47. 機能動詞構文のマイニング

動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．

学習を行う	に を	元に 経験を

In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

    def __str__(self):
        return f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}"

class Chunk:
    def __init__(self, morphs=None, dst=-1):
        self.morphs = morphs if morphs else []  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst  # 係り先文節インデックス番号
        self.srcs = []  # 係り元文節インデックス番号のリスト

    def __str__(self):
        morphs_surface = ''.join([morph.surface for morph in self.morphs])
        return f"Chunk: {morphs_surface}, dst: {self.dst}, srcs: {self.srcs}"

    def get_text(self):
        return ''.join([morph.surface for morph in self.morphs if morph.pos != '記号'])

    def get_pos(self, pos):
        return [morph for morph in self.morphs if morph.pos == pos]

    def get_sahen_wo(self):
        for i in range(len(self.morphs) - 1):
            if self.morphs[i].pos1 == 'サ変接続' and self.morphs[i + 1].surface == 'を' and self.morphs[i + 1].pos == '助詞':
                return ''.join([self.morphs[i].surface, self.morphs[i + 1].surface])
        return None

def parse_cabocha_output(parsed_file):
    sentences = []
    with open(parsed_file, 'r', encoding='utf-8') as f:
        chunks = {}
        chunk = None
        for line in f:
            if line == 'EOS\n':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                if chunks:
                    sorted_chunks = sorted(chunks.items())
                    sentence = [chunk for idx, chunk in sorted_chunks]
                    for idx, chunk in sorted_chunks:
                        if chunk.dst != -1:
                            sentence[chunk.dst].srcs.append(idx)
                    sentences.append(sentence)
                chunks = {}
                chunk = None
            elif line[0] == '*':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                cols = line.split(' ')
                chunk_idx = int(cols[1])
                dst = int(cols[2].rstrip('D'))
                chunk = Chunk(dst=dst)
            else:
                surface, feature = line.split('\t')
                features = feature.split(',')
                if len(features) >= 7:
                    morph = Morph(surface, features[6], features[0], features[1])
                    chunk.morphs.append(morph)
    return sentences

def extract_sahen_case_patterns(parsed_file, output_file):
    sentences = parse_cabocha_output(parsed_file)
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            for chunk in sentence:
                sahen_wo = chunk.get_sahen_wo()
                if sahen_wo:
                    dst_chunk = sentence[chunk.dst]
                    verbs = dst_chunk.get_pos('動詞')
                    if verbs:
                        verb = verbs[0].base
                        predicate = sahen_wo + verb
                        particles = []
                        phrases = []
                        for src in dst_chunk.srcs:
                            src_chunk = sentence[src]
                            particles += [morph.surface for morph in src_chunk.morphs if morph.pos == '助詞']
                            phrases.append(src_chunk.get_text())
                        if particles:
                            particles = sorted(set(particles))
                            phrases = [phrase for _, phrase in sorted(zip(particles, phrases))]
                            particles = sorted(set(particles))
                            f.write(f"{predicate}\t{' '.join(particles)}\t{' '.join(phrases)}\n")

# 指定解析結果ファイルのパス
parsed_file = 'ai.ja.txt.parsed'
# 出力ファイルのパス
output_file = 'sahen_case_patterns.txt'

# サ変接続名詞を含む動詞の格パターンと項を抽出してファイルに保存
extract_sahen_case_patterns(parsed_file, output_file)
